# Model Fit in Linear Regression - Lab

## Introduction
In this lab, you'll learn how to evaluate your model results, and you'll learn methods to select the appropriate features using stepwise selection.

## Objectives
You will be able to:
* Analyze the results of regression and R-squared and adjusted R-squared 
* Understand and apply forward and backward predictor selection

## The Boston Housing Data once more

We pre-processed the Boston Housing data the same way we did before:

- We dropped `'ZN'` and `'NOX'` completely
- We categorized `'RAD'` in 3 bins and `'TAX'` in 4 bins
- We transformed `'RAD'` and `'TAX'` to dummy variables and dropped the first variable
- We used min-max-scaling on `'B'`, `'CRIM'`, and `'DIS'` (and logtransformed all of them first, except `'B'`)
- We used standardization on `'AGE'`, `'INDUS'`, `'LSTAT'`, and `'PTRATIO'` (and logtransformed all of them first, except for `'AGE'`) 

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns=boston.feature_names)
boston_features = boston_features.drop(['NOX', 'ZN'], axis=1)

# First, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0, 6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# First, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix='TAX', drop_first=True)
rad_dummy = pd.get_dummies(bins_rad, prefix='RAD', drop_first=True)
boston_features = boston_features.drop(['RAD', 'TAX'], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

age = boston_features['AGE']
b = boston_features['B']
logcrim = np.log(boston_features['CRIM'])
logdis = np.log(boston_features['DIS'])
logindus = np.log(boston_features['INDUS'])
loglstat = np.log(boston_features['LSTAT'])
logptratio = np.log(boston_features['PTRATIO'])

# Min-Max scaling
boston_features['B'] = (b-min(b))/(max(b)-min(b))
boston_features['CRIM'] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features['DIS'] = (logdis-min(logdis))/(max(logdis)-min(logdis))

# Standardization
boston_features['AGE'] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features['INDUS'] = (
    logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features['LSTAT'] = (
    loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features['PTRATIO'] = (
    logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

## Perform stepwise selection

The function for stepwise selection is copied below. Use this function provided on your preprocessed Boston Housing data.

In [2]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [8]:
selected = stepwise_selection(boston_features, boston.target, verbose=True)
print(selected)

Add  LSTAT                          with p-value 9.27989e-122
Add  RM                             with p-value 1.98621e-16
Add  PTRATIO                        with p-value 2.5977e-12
Add  DIS                            with p-value 2.85496e-09
Add  B                              with p-value 2.77572e-06
Add  INDUS                          with p-value 0.0017767
Add  CHAS                           with p-value 0.0004737
['LSTAT', 'RM', 'PTRATIO', 'DIS', 'B', 'INDUS', 'CHAS']


### Build the final model again in Statsmodels

In [13]:
Y = boston.target
X = boston_features[selected]
X_w_const = sm.add_constant(X)
sel_model = sm.OLS(Y, X_w_const).fit()
sel_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.770
Method:                 Least Squares   F-statistic:                     242.7
Date:                Sun, 20 Oct 2019   Prob (F-statistic):          4.89e-156
Time:                        11:22:13   Log-Likelihood:                -1464.7
No. Observations:                 506   AIC:                             2945.
Df Residuals:                     498   BIC:                             2979.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.0123      2.829      1.772      0.077      -0.545      10.570
LSTAT         -5.6444      0.320    -17.629      0.000      -6.274      -5.015
RM             2.8712      0.388      7.405      0.000       2.109       3.633
PTRATIO       -1.3564      0.227     -5.983      0.000      -1.802      -0.911
DIS           -9.7229      1.326     -7.333      0.000     -12.328      -7.118
B              4.0619      0.934      4.347      0.000       2.226       5.898
INDUS         -1.2099      0.334     -3.619      0.000      -1.867      -0.553
CHAS           2.7988      0.795      3.519      0.000       1.236       4.362
==============================================================================
Omnibus:                      105.185   Durbin-Watson:                   1.099
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              423.621
Skew:                           0.878   Prob(JB):                     1.03e-92
Kurtosis:                       7.124   Cond. No.                         96.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The stepwise procedure mentions that `'INDUS'` was added with a p-value of 0.0017767, but our statsmodels output returns a p-value of 0.000. Use some of the stepwise procedure logic to find the intuition behind this!

#### This must be because the p-values of other parameter extimates change while new variables are being added/removed during the stepwise prodecure.

## Use Feature ranking with recursive feature elimination

Use feature ranking to select the 5 most important features

In [14]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [15]:
linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select=5)
selector = selector.fit(boston_features, boston.target)
display(selector.support_)
display(selector.ranking_)

array([False, False,  True,  True, False,  True, False,  True,  True,
       False, False, False])

array([4, 7, 1, 1, 8, 1, 3, 1, 1, 5, 6, 2])

Fit the linear regression model again using the 5 selected columns

In [19]:
selected_x = boston_features.columns[selector.support_]

linreg.fit(boston_features[selected_x], boston.target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Now, predict $\hat y$ using your model. You can use `.predict()` in scikit-learn. 

In [21]:
y_hat = linreg.predict(boston_features[selected_x])
y_hat

array([31.16165902, 23.76929453, 34.8764376 , 36.98692436, 31.32930715,
       29.06938762, 18.83320099, 14.70915944,  8.01751688, 14.90750061,
       14.49961186, 17.84334784, 15.96671008, 23.35014491, 21.5475942 ,
       22.80420786, 25.67823103, 17.66801955, 17.36837517, 19.86812537,
       12.67589909, 18.42874828, 15.95977353, 14.09380242, 16.3420057 ,
       13.99038842, 16.58165948, 15.0909717 , 20.71269856, 22.06463506,
       11.91943166, 19.11352766,  9.2779531 , 14.33996426, 13.34927881,
       22.57757018, 20.30738333, 22.88337432, 21.8018023 , 31.92538034,
       41.52527107, 31.06370984, 27.07153202, 24.77046631, 21.62254369,
       20.00275986, 16.96128095, 14.4887785 ,  7.13719738, 14.42912675,
       17.2790188 , 21.48105026, 28.91702238, 22.28080957, 15.84257515,
       31.73722975, 26.72444484, 32.38609452, 24.47989229, 21.05648509,
       16.58337781, 16.34295586, 26.03847483, 23.20882414, 25.68084118,
       29.5199626 , 19.61369356, 22.40808622, 16.44091927, 21.58

Now, using the formulas of R-squared and adjusted R-squared below, and your Python/numpy knowledge, compute them and contrast them with the R-squared and adjusted R-squared in your statsmodels output using stepwise selection. Which of the two models would you prefer?

$SS_{residual} = \sum (y - \hat{y})^2 $

$SS_{total} = \sum (y - \bar{y})^2 $

$R^2 = 1- \dfrac{SS_{residual}}{SS_{total}}$

$R^2_{adj}= 1-(1-R^2)\dfrac{n-1}{n-p-1}$

In [22]:
ss_res = np.sum((boston.target - y_hat)**2)
ss_tot = np.sum((boston.target - np.mean(boston.target))**2)

r2 = 1 - (ss_res/ss_tot)
adj_r2 = 1 - (1 - r2)*((len(boston.target) - 1)/(len(boston.target) - len(selected_x) - 1))

print(r2)
print(adj_r2)
# r_squared is 0.742981
# adjusted_r_squared is 0.740411

0.7429807743129864
0.7404105820561162


## Level up (Optional)

- Perform variable selection using forward selection, using this resource: https://planspace.org/20150423-forward_selection_with_statsmodels/. Note that this time features are added based on the adjusted R-squared!
- Tweak the code in the `stepwise_selection()` function written above to just perform forward selection based on the p-value 

## Summary
Great! You now performed your own feature selection methods!